In [3]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time

# third-party imports
import numpy as np
import pandas as pd
import requests
from requests.exceptions import SSLError

# customisations - ensure tables show all columns
pd.set_option("display.max_columns", 100)

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
def get_request(url,parameters=None, steamspy=False):
    """Return json-formatted response of a get request using optional parameters.
    
    Parameters
    ----------
    url : string
    parameters : {'parameter': 'value'}
        parameters to pass as part of get request
    
    Returns
    -------
    json_data
        json-formatted response (dict-like)
    """
    try:
        response = requests.get(url=url, params=parameters)
    except SSLError as s:
        print('SSL Error:', s)
        
        for i in range(5, 0, -1):
            print('\rWaiting... ({})'.format(i), end='')
            time.sleep(1)
        print('\rRetrying.' + ' '*10)
        
        # recursively try again
        return get_request(url, parameters, steamspy)
    
    if response:
        try:
            return response.json()
        except:
            False
    else:
        # We do not know how many pages steamspy has... and it seems to work well, so we will use no response to stop.
        if steamspy:
            return "stop"
        else :
            # response is none usually means too many requests. Wait and try again 
            print('No response, waiting 10 seconds...')
            time.sleep(10)
            print('Retrying.')
            return get_request(url, parameters, steamspy)

Define Download Data

In [20]:
def get_app_data(app_list, start, stop, parser, pause):
    """Return list of app data generated from parser.
    
    parser : function to handle request
    """
    app_data = []
    
    # iterate through each row of app_list, confined by start and stop
    for index, appid in app_list[start:stop].items():
        print('Current index: {}'.format(index), end='\r')

        # retrive app data for a row, handled by supplied parser, and append to list
        try:
            data = parser(appid)
            app_data.append(data)
        except:
            print("Error with "+str(appid))
        time.sleep(pause) # prevent overloading api with requests
    
    return app_data


def process_batches(parser, app_list, download_path, data_filename, index_filename,
                    columns, begin=0, end=-1, batchsize=100, pause=1):
    """Process app data in batches, writing directly to file.
    
    parser : custom function to format request
    app_list : dataframe of appid and name
    download_path : path to store data
    data_filename : filename to save app data
    index_filename : filename to store highest index written
    columns : column names for file
    
    Keyword arguments:
    
    begin : starting index (get from index_filename, default 0)
    end : index to finish (defaults to end of app_list)
    batchsize : number of apps to write in each batch (default 100)
    pause : time to wait after each api request (defualt 1)
    
    returns: none
    """
    print('Starting at index {}:\n'.format(begin))
    
    # by default, process all apps in app_list
    if end == -1:
        end = len(app_list) + 1
    
    # generate array of batch begin and end points
    batches = np.arange(begin, end, batchsize)
    batches = np.append(batches, end)
    
    apps_written = 0
    batch_times = []
    
    for i in range(len(batches) - 1):
        start_time = time.time()
        
        start = batches[i]
        stop = batches[i+1]
        
        app_data = get_app_data(app_list, start, stop, parser, pause)
        
        rel_path = os.path.join(download_path, data_filename)
        
        # writing app data to file
        with open(rel_path, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=columns, extrasaction='ignore')
            
            for j in range(3,0,-1):
                print("\rAbout to write data, don't stop script! ({})".format(j), end='')
                time.sleep(0.5)
            
            writer.writerows(app_data)
            print('\rExported lines {}-{} to {}.'.format(start, stop-1, data_filename), end=' ')
            
        apps_written += len(app_data)
        
        idx_path = os.path.join(download_path, index_filename)
        
        # writing last index to file
        with open(idx_path, 'w') as f:
            index = stop
            print(index, file=f)
            
        # logging time taken
        end_time = time.time()
        time_taken = end_time - start_time
        
        batch_times.append(time_taken)
        mean_time = statistics.mean(batch_times)
        
        est_remaining = (len(batches) - i - 2) * mean_time
        
        remaining_td = dt.timedelta(seconds=round(est_remaining))
        time_td = dt.timedelta(seconds=round(time_taken))
        mean_td = dt.timedelta(seconds=round(mean_time))
        
        print('Batch {} time: {} (avg: {}, remaining: {})'.format(i, time_td, mean_td, remaining_td))
            
    print('\nProcessing batches complete. {} apps written'.format(apps_written))

In [21]:
def reset_index(download_path, index_filename):
    """Reset index in file to 0."""
    rel_path = os.path.join(download_path, index_filename)
    
    f= open(rel_path, 'w')
    f.write("0")
        

def get_index(download_path, index_filename):
    """Retrieve index from file, returning 0 if file not found."""
    try:
        rel_path = os.path.join(download_path, index_filename)

        with open(rel_path, 'r') as f:
            index = int(f.readline())
            #This just reads the initial line
    
    except FileNotFoundError:
        index = 0
        
    return index


def prepare_data_file(download_path, filename, index, columns):
    """Create file and write headers if index is 0."""
    if index == 0:
        rel_path = os.path.join(download_path, filename)

        with open(rel_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=columns)
            writer.writeheader()

Download Steam Data

In [22]:
def getAppListBatch(url, parameters):
    json_data = get_request(url, parameters=parameters)
    steam_id = pd.DataFrame.from_dict(json_data["response"]["apps"])
    try:
        more_results = json_data["response"]["have_more_results"]
        last_appid =  json_data["response"]["last_appid"]
    except:
        more_results = False
        last_appid = False
    return more_results, steam_id, last_appid

def get_update_ids_old(updatedlist, oldlist):
    updatedlist['key1'] = 1
    oldlist['key2'] = 1
    updatedlist = pd.merge(updatedlist, oldlist, right_on=['steam_appid','name'],left_on=['appid','name'], how = 'outer')
    updatedlist = updatedlist[~(updatedlist.key2 == updatedlist.key1)]
    updatedlist = updatedlist.drop(['key1','key2','steam_appid'], axis=1)
    return updatedlist

def get_update_ids(idList, oldFullList):
    #We are going to forget about names and only care about IDs.
    idList = idList["appid"]
    oldFullList = oldFullList["steam_appid"]
    oldFullList.columns = ["appid"]
    updatedList = pd.concat([idList, oldFullList])
    updatedList = updatedList.drop_duplicates(keep=False)
    updatedList = updatedList.reset_index(drop=True)
    return updatedList

In [23]:
def getAppList():
    with open('../data/steam_key.txt') as f:
        key = f.read()

    url = "https://api.steampowered.com/IStoreService/GetAppList/v1/?"
    parameters = {"key": key}
    more_results = True
    begin = True
    # from the request we get the more_results flag and also the last_appid, so we use them for the next requests.
    while (more_results):
        more_results, steam_ids, last_appid = getAppListBatch(url, parameters)
        parameters["last_appid"] = last_appid
        if (begin):
            steam_allids = steam_ids
            begin = False
        else:
            steam_allids = pd.concat([steam_allids, steam_ids])
    return steam_allids
# request 'all' from steam spy and parse into dataframe

In [24]:
def parse_steam_request(appid):
    """Unique parser to handle data from Steam Store API.
    
    Returns : json formatted data (dict-like)
    """
    with open('../data/steam_key.txt') as f:
        key = f.read()
        
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": appid, "key": key}
    
    json_data = get_request(url, parameters=parameters)
    json_app_data = json_data[str(appid)]
    
    if json_app_data['success']:
        data = json_app_data['data']
    else:
        data = {'steam_appid': appid}
        
    return data


# Set file parameters
download_path = '../data/download/'
steam_app_data = 'steam_app_data.csv'
steam_app_data_delta = 'steam_app_data_delta.csv'
steam_index = 'steam_index.txt'

steam_columns = [
    'type', 'name', 'steam_appid', 'required_age', 'is_free', 'controller_support',
    'dlc', 'detailed_description', 'about_the_game', 'short_description', 'fullgame',
    'supported_languages', 'header_image', 'website', 'pc_requirements', 'mac_requirements',
    'linux_requirements', 'legal_notice', 'drm_notice', 'ext_user_account_notice',
    'developers', 'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
    'platforms', 'metacritic', 'reviews', 'categories', 'genres', 'screenshots',
    'movies', 'recommendations', 'achievements', 'release_date', 'support_info',
    'background', 'content_descriptors'
]

# Overwrites last index for demonstration (would usually store highest index so can continue across sessions)
if (os.path.isfile(download_path+steam_app_data_delta) == False):
    reset_index(download_path, steam_index)

# Retrieve last index downloaded from file
index = get_index(download_path, steam_index)

# Wipe or create data file and write headers if no previous  data
if (os.path.isfile(download_path+steam_app_data) == False):
    prepare_data_file(download_path, steam_app_data, index, steam_columns)
    
# Wipe or create data file delta and write headers if index is 0
if (os.path.isfile(download_path+steam_app_data_delta) == False):
    prepare_data_file(download_path, steam_app_data_delta, index, steam_columns)
    
    
# Here we get the list of appids from steam
full_steam_ids = getAppList()

# Here we get the real list of ids not yet in our dataframe. If this is the first time we are downloading the data, we can skip
# This step and instead use the full app_list.
try:
    oldlist = pd.read_csv('../data/download/steam_app_data.csv', usecols = ['name','steam_appid'])
    steam_ids = get_update_ids(full_steam_ids, oldlist)
except FileNotFoundError:
    print("Pre-existing file not found. First time downloading full app data from steam. This will take a while.\n")
    steam_ids = full_steam_ids

In [25]:
print("New IDs detected: "+str(len(steam_ids)))

New IDs detected: 98460


In [26]:
# I separated the long process to be able to debug it better.
# Set end and chunksize for demonstration - remove to run through entire app list
# Here by default we passed "app_list" that contained all the information and saved it, now we will modify it a bit
# And add pre-processing and post-processing
print("Adding "+str(len(steam_ids))+" new ids.\n")
process_batches(
    parser=parse_steam_request,
    app_list=steam_ids,
    download_path=download_path,
    data_filename=steam_app_data_delta,
    index_filename=steam_index,
    columns=steam_columns,
    begin=index,
    #end=10,
    #batchsize=5
)

try:
    oldlist = pd.read_csv('../data/download/steam_app_data.csv')
    # We change the old file to backup, so remove any backup named this way before...
    os.rename('../data/download/steam_app_data.csv', '../data/download/steam_app_data_backup.csv')
    newlist = pd.read_csv('../data/download/steam_app_data_delta.csv')
    oldlist = oldlist.append(newlist, ignore_index=True)
    oldlist.to_csv('../data/download/steam_app_data.csv', index=False)
except FileNotFoundError:
    os.rename('../data/download/steam_app_data_delta.csv', '../data/download/steam_app_data.csv')

Adding 98460 new ids.

Starting at index 0:

Exported lines 0-99 to steam_app_data_delta.csv. Batch 0 time: 0:02:35 (avg: 0:02:35, remaining: 1 day, 18:26:54)
Exported lines 100-199 to steam_app_data_delta.csv. Batch 1 time: 0:02:35 (avg: 0:02:35, remaining: 1 day, 18:24:03)
Exported lines 200-299 to steam_app_data_delta.csv. Batch 2 time: 0:02:35 (avg: 0:02:35, remaining: 1 day, 18:20:39)
Exported lines 300-399 to steam_app_data_delta.csv. Batch 3 time: 0:02:36 (avg: 0:02:35, remaining: 1 day, 18:20:20)
Exported lines 400-499 to steam_app_data_delta.csv. Batch 4 time: 0:02:38 (avg: 0:02:36, remaining: 1 day, 18:25:51)
Exported lines 500-599 to steam_app_data_delta.csv. Batch 5 time: 0:02:38 (avg: 0:02:36, remaining: 1 day, 18:29:51)
Exported lines 600-699 to steam_app_data_delta.csv. Batch 6 time: 0:02:37 (avg: 0:02:36, remaining: 1 day, 18:30:04)
Exported lines 700-799 to steam_app_data_delta.csv. Batch 7 time: 0:02:40 (avg: 0:02:37, remaining: 1 day, 18:35:15)
Exported lines 800-899

C:\Users\jdejr\AppData\Local\Temp\ipykernel_58132\127405560.py:22: DtypeWarning: Columns (3,4,6,18,19,23,24,27,28,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  newlist = pd.read_csv('../data/download/steam_app_data_delta.csv')


AttributeError: 'DataFrame' object has no attribute 'append'

In [28]:
os.rename('../data/download/steam_app_data_delta.csv', '../data/download/steam_app_data.csv')

In [29]:
steam_app_data = pd.read_csv('../data/download/steam_app_data.csv')

C:\Users\jdejr\AppData\Local\Temp\ipykernel_58132\3041477814.py:1: DtypeWarning: Columns (3,4,6,18,19,23,24,27,28,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  steam_app_data = pd.read_csv('../data/download/steam_app_data.csv')


In [30]:
steam_app_data

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,Counter-Strike,10,0.0,False,NaN,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,NaN,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'USD', 'initial': 999, 'final': 9...","[574941, 7]","[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 146647},NaN,"{'coming_soon': False, 'date': 'Nov 1, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,game,Team Fortress Classic,20,0.0,False,NaN,NaN,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,NaN,"English, French, German, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'USD', 'initial': 499, 'final': 4...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 5888},NaN,"{'coming_soon': False, 'date': 'Apr 1, 1999'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
2,game,Day of Defeat,30,0.0,False,NaN,NaN,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,NaN,"English, French, German, Italian, Spanish - Spain",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'USD', 'initial': 499, 'final': 4...","[30, 944613]","[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 3937},NaN,"{'coming_soon': False, 'date': 'May 1, 2003'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
3,game,Deathmatch Classic,40,0.0,False,NaN,NaN,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,NaN,"English, French, German, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{

In [31]:
steam_app_data[steam_app_data.duplicated(subset="steam_appid")]

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
655,game,Tom Clancy's Splinter Cell Conviction™ Deluxe ...,33220,17.0,False,NaN,[33372],An investigation into his daughter’s death unw...,An investigation into his daughter’s death unw...,An investigation into his daughter’s death unw...,NaN,"English, French, German, Italian, Spanish - Spain",https://cdn.akamai.steamstatic.com/steam/apps/...,http://splintercell.us.ubi.com/conviction/,{'minimum': '<strong>Minimum:</strong><br>\t\t...,"{'minimum': '<ul class=""bb_ul"">Does not suppor...",[],HIGH SPEED INTERNET ACCESS AND CREATION OF A U...,NaN,Ubisoft account required (Supports Linking to ...,['Ubisoft Montreal'],['Ubisoft'],NaN,"{'currency': 'USD', 'initial': 1999, 'final': ...","[4261, 4158]","[{'name': 'default', 'title': ""Buy Tom Clancy'...","{'windows': True, 'mac': False, 'linux': False}","{'score': 83, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 3613},NaN,"{'coming_soon': False, 'date': 'Apr 29, 2010'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
25052,game,The Elder Scrolls IV: Oblivion® Game of the Ye...,22330,17.0,False,NaN,NaN,The Elder Scrolls IV: Oblivion® Game of the Ye...,The Elder Scrolls IV: Oblivion® Game of the Ye...,The Elder Scrolls IV: Oblivion® Game of the Ye...,NaN,English,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.elderscrolls.com,"{'minimum': '<ul class=""bb_ul""><li><strong>OS ...",[],[],The Elder Scrolls IV: Oblivion® Game of the Ye...,NaN,NaN,['Bethesda Game Studios®'],['Bethesda Softworks'],NaN,"{'currency': 'USD', 'initial': 1999, 'final': ...","[1679, 1678]","[{'name': 'default', 'title': 'Buy The Elder S...","{'windows': True, 'mac': False, 'linux': False}","{'score': 94, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 2, 'description': 'Single-player'}, {'...",NaN,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...","[{'id': 5286, 'name': 'The Elder Scrolls IV: O...",{'total': 35500},NaN,"{'coming_soon': False, 'date': 'Jun 16, 2009'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
25055,game,Earthworm Jim,38480,0.0,False,NaN,NaN,Jim wasn't always a studly super-worm. He was ...,Jim wasn't always a studly super-worm. He was ...,Jim wasn't always a studly super-worm. He was ...,NaN,English,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",[],[],NaN,NaN,NaN,['Shiny Entertainment'],['Interplay Inc.'],NaN,"{'currency': 'USD', 'initial': 1999, 'final': ...",[2382],"[{'name': 'default', 'title': 'Buy Earthworm J...","{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 2, 'description': 'Single-player'}]","[{'id': '1', 'description': 'Action'}, {'id': ...","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 131},NaN,"{'coming_soon': False, 'date': 'Nov 4, 2009'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
25056,game,Sam & Max: The Devil’s Playhouse,31220,0.0,False,NaN,NaN,"<p>5 Games, 1 Price! Delivered monthly</p>\n\t...","<p>5 Games, 1 Price! Delivered monthly</p>\n\t...","5 Games, 1 Price! Delivered monthly Wield psyc...",NaN,English,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.samandmaxgames.com/thedevilsplayhouse,{'minimum': '<ul>\n\t\t\t\t\t\t\t\t\t<li><stro...,{'m

In [32]:
steam_app_data[steam_app_data["steam_appid"] == 34330]

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
1297,game,Total War: SHOGUN 2,34330,0.0,False,NaN,"[223180, 201279, 201277, 34348, 34342, 34343, ...",<h1>Total War: SHOGUN 2 out now for Linux.</h1...,<strong>MASTER THE ART OF WAR</strong><br>\t\t...,Total War: SHOGUN 2 is the perfect mix of real...,NaN,"English<strong>*</strong>, Czech, French<stron...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.totalwar.com/shogun2,{'minimum': '<strong>Minimum:</strong><br>\t\t...,"{'minimum': '<ul class=""bb_ul""><li><strong>OS:...",{'minimum': '<strong>Minimum:</strong><br><ul ...,© SEGA. SEGA and the SEGA logo are registered ...,NaN,NaN,"['CREATIVE ASSEMBLY', 'Feral Interactive (Mac)...","['SEGA', 'Feral Interactive (Mac)', 'Feral Int...","[{'appid': 34350, 'description': ''}]","{'currency': 'USD', 'initial': 2999, 'final': ...","[7587, 18408]","[{'name': 'default', 'title': 'Buy Total War: ...","{'windows': True, 'mac': True, 'linux': True}","{'score': 90, 'url': 'https://www.metacritic.c...",“...the best gameplay and design of any entry ...,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '2', 'description': 'Strategy'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 31187},"{'total': 106, 'highlighted': [{'name': 'Stran...","{'coming_soon': False, 'date': 'Mar 15, 2011'}","{'url': 'https://support.sega.co.uk', 'email':...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"


In [34]:
len(steam_app_data)-len(full_steam_ids)

-120

In [35]:
diff_ids = get_update_ids(full_steam_ids, steam_app_data)

In [36]:
len(diff_ids)

135